In [ ]:
!pip install tensorflow

In [ ]:
!pip install sentencepiece

In [ ]:
# You should have the rest already installed

In [1]:
import os
import re
import string
import pandas as pd
import numpy as np
import emoji
import contractions
import html
import translate
import sentencepiece
import torch.nn as nn
import torch
import tensorflow
import multiprocessing

from spellchecker import SpellChecker

from translate import Translator
from emoji import demojize

from collections import Counter
from textblob import TextBlob

from transformers import AutoTokenizer
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from transformers import XLNetForSequenceClassification, XLNetTokenizer

from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

# Set the maximum width for column display to a large value (e.g., 200 characters)
pd.set_option('display.max_colwidth', 200)




In [2]:

# Change directory
print(os.getcwd())
os.chdir('C://Users/andre/Job Portfolio Projects/DataGlacierVI/sentiment.analysis/')
print(os.getcwd())


C:\Users\andre\Job Portfolio Projects\DataGlacierVI\sentiment.analysis
C:\Users\andre\Job Portfolio Projects\DataGlacierVI\sentiment.analysis


In [3]:

dftrt = pd.read_csv('train_tweets.csv')
# dftt = pd.read_csv('test_tweets.csv')


In [ ]:
display(dftt.head())
display(dftt.info())
display(dftt.shape)

In [ ]:
display(dftrt.head())
display(dftrt.info())
display(dftrt.shape)

In [ ]:
# Percentage of original dataset that test data is
dftt.shape[0]/(dftt.shape[0] + dftrt.shape[0])

In [4]:
# Average tweet length in characters

def avg_num_chars(df, colname):
    
    # Create empty list
    lst = []
    # For loop to iterate through every tweet
    for tweet in df[colname]:
        
        # Get len of every string
        length = len(tweet)
        
        # Append the length to a list
        lst.append(length)

    # Sum all elements in list
    len_sum = sum(lst)
    
    # Divide 'len_sum' by length of list
    print('Average tweet length: %d characters' % (len_sum/len(lst)))
    print('With a standard deviation of: %d characters per tweet' % (np.std(lst)), '\n')
    # Return result
    return len_sum/len(lst), np.std(lst)
    

In [5]:

def avg_num_wrds(df, colname):
    
    # Lst to capture lengths of wrd_lsts
    lst = []
    
    # Sum variable
    word_sum = 0
    
    # Loop for counting words
    for tweet in df[colname]:
        
        # Split tweet according to white space
        word_lst = tweet.split()
        
        # Find length of split lst
        num_wrds = len(word_lst)
        
        # Append lengths of num_wrds in each lst
        lst.append(num_wrds)
        
        # Sum the wrds of the tweet
        word_sum += num_wrds
        
    # Divide by the number of tweets to get average num wrds
    avg_wrds_in_tweet = word_sum / len(df[colname])
    
    # Print results
    print('Average number of words per tweet: %d' % (avg_wrds_in_tweet))
    print('Standard deviation of the number of words: %d' % (np.std(lst)), '\n')
    
    # Return result
    return avg_wrds_in_tweet, np.std(lst)
        
        
    

In [ ]:

# Number of tweets in training set (pre-cleaning)
num_tweets = len(dftrt)
num_tweets


In [6]:

# Averages of number of chars, words, in tweets. Plus standard deviations (pre-cleaning)

avg_and_stdvs = avg_num_chars(dftrt, 'tweet')[0]/avg_num_wrds(dftrt, 'tweet')[0]
print('Average length of a word in a tweet: %d' % (avg_and_stdvs))


Average tweet length: 84 characters
With a standard deviation of: 29 characters per tweet 

Average number of words per tweet: 13
Standard deviation of the number of words: 5 

Average length of a word in a tweet: 6


In [9]:

def clean_tweet(tweet):
    
    # Instantiate the translator
    translator = Translator(to_lang='en')
    
    # Translate to english
    tweet = translator.translate(tweet)
   
    # Convert to Lowercase
    tweet = tweet.lower()
    
    # Fix contractions
    tweet = contractions.fix(tweet)
    
    tweet = emoji.demojize(tweet)
    
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    
    # Remove user mentions like: '@username'
    tweet = re.sub(r'@\w+', '', tweet)
    
    # Remove special characters and punctuation (except for spaces)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    
    # Remove numbers
    tweet = re.sub(r'[0-9]', '', tweet)
    
    # Remove all non-ascii chars
    tweet = ''.join(char for char in tweet if ord(char) < 128)
    
    # Remove extra spaces
    tweet = ' '.join(tweet.split())
    
    # Remove html
    tweet = html.unescape(tweet)
    
    
    return tweet


In [ ]:

# Nulls pre-cleaning
rows_with_null = dftrt.isnull().sum()
rows_with_null


In [10]:

# Cleans tweets of punctuation, special characters, url's, user mentions, numbers, extra spaces, emoji's, html

dftrt['clean_tweet'] = dftrt['tweet'].apply(clean_tweet)
dftrt.head()


,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit i cannot use because they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,model i love you take with you all the time in you are
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [11]:
# Avg chars, words, and avg lenght of word in tweet (post-cleaning)

avg_and_stdvs = avg_num_chars(dftrt, 'clean_tweet')[0]/avg_num_wrds(dftrt, 'clean_tweet')[0]
print('Average length of a word in a tweet: %d' % (avg_and_stdvs))

Average tweet length: 70 characters
With a standard deviation of: 28 characters per tweet 

Average number of words per tweet: 12
Standard deviation of the number of words: 5 

Average length of a word in a tweet: 5


In [ ]:

nulls_after_cleaning = dftrt.isnull().sum()
nulls_after_cleaning


In [ ]:
def spell_check_batch(texts):
    # Spell check a batch of texts
    return [spell.correction(text) for text in texts]

In [ ]:

# Spell checker
def spell_check(tweet):
    
    # Tokenize the tweet
    words = tweet.split()
    # Track original and corrected words
    original_tweet = []
    corrected_tweet = []
    
    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word is not None: 
            corrected_tweet.append(corrected_word)
        else:
            corrected_tweet.append(word)  # Keep the original word if no correction found
        original_tweet.append(word)
        
    # Join the corrected words back into a sentence
    corrected_tweet_text = ' '.join(corrected_tweet)
    original_tweet_text = ' '.join(original_tweet)
    
    
    return original_tweet_text, corrected_tweet_text
    

In [ ]:
# Initialize the spell checker and word frequency objects
spell = SpellChecker()

# Split your dataset into batches
batch_size = 100

batch = [dftrt['clean_tweet'][i:i+batch_size] for i in range(0, len(dftrt), batch_size)]

# Initialize a multiprocessing pool
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())

# 
corrected_batch = pool.map(spell_check_batch, batch)

# Combine the results into a single list
corrected_tweet = [text for batch in corrected_batches for text in batch]

In [ ]:
spell = SpellChecker()

print(spell.correction('sme'))

In [ ]:
dftrt['spell_checked_tweet'] = dftrt['clean_tweet'].apply(spell_check)

In [ ]:

na_after_cleaning = dftrt.isna().sum()
na_after_cleaning


In [ ]:

### > Splitting the Data: Using the 'train_tweets' for training, val, & testing

# Shuffle the training, validation, & testing data
shuffled_data = dftrt.sample(frac=1, random_state=42)

# Instantiate ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate size of each dataset
train_size = int(len(shuffled_data) * train_ratio)
val_size = int(len(shuffled_data) * val_ratio)
test_size = int(len(shuffled_data) * test_ratio)

# Split the data into training, validation, and test sets
train_data = shuffled_data[:train_size]
val_data = shuffled_data[train_size:(train_size + val_size)]
test_data = shuffled_data[(train_size + val_size):(train_size + val_size + test_size)]

# Ensure order is preserved within each set
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)


In [ ]:

# Load and preprocess training data
def preprocess_data(df, tokenizer, max_len=110, apply_smote=False, test_data=False):

    # Clean the tweets of the training set
    df['clean_tweet'] = df['tweet'].apply(clean_tweet)

    # Create an instance of the tokenizer
    tokenizer_instance = tokenizer.from_pretrained("xlnet-base-cased")

    # List to hold text tokens: ["Hello", ",", "world", "!", "[SEP]"]
    tokenized_texts = []

    # Tokenize the tweets: try-block to identify any problem tokens
    for tweet in df['clean_tweet']:

        try:

          # Convert the token IDs to strings: special tokens = [SEP] etc..
          # str(token_id): casting int to str
          # is_split_into_words: splits tokens into words or subwords
          # Ex: [[CLS], 'un', 'happiness', [CLS]] <- 'unhappiness' split

            tokens = [str(token_id) for token_id in tokenizer_instance.encode(str(tweet), add_special_tokens=True, is_split_into_words=True)]

        except Exception as e:

            print(f"Error converting tokens to IDs: {e}")
            print(f"Problematic text: {tweet}")

            continue

        # Append the tokens to list: ["Hello", ",", "world", "!", "[SEP]"]
        tokenized_texts.append(tokens)


    # List to hold input ID's: [1045, 2293, 3019, 3601, 2653, 6360]
    input_ids_list = []

    # List to hold attention masks: [1, 1, 1, 0, 0, 0]
    attention_masks = []

    # Truncate text sequences if necessary
    for tokens in tokenized_texts:

        # Tokenize and pad input_ids: each token gets mapped to a unique int ID
        # if sequence is < 100, then 0's to fill the missing elements
        input_id = tokenizer_instance.convert_tokens_to_ids(tokens)
        input_id += [0] * (max_len - len(input_id))
        input_ids_list.append(input_id)

        # Create attention mask (1 for real tokens, 0 for padding)
        attention_mask = [1] * len(tokens) + [0] * (max_len - len(tokens))

        # Pad attention_mask to match max_len (100)
        attention_mask += [0] * (max_len - len(attention_mask))
        attention_masks.append(attention_mask)

    input_ids = torch.tensor(input_ids_list)
    attention_masks = torch.tensor(attention_masks)
    y_label = torch.tensor(df['label'])


    if apply_smote:

        smote = SMOTE(sampling_strategy='auto', random_state=42)
        input_ids_resampled, y_resampled = smote.fit_resample(input_ids, y_label)

        y_resampled = torch.tensor(y_resampled)
        input_ids_resampled = torch.tensor(input_ids_resampled)

        # Calculate length of resampled y_labels
        max_samples = len(y_resampled)

        # Pad attention_masks to match the original max_samples
        attention_masks = torch.cat([attention_masks, torch.zeros(max_samples-len(attention_masks), max_len)])

        # Cast attention masks to a tensor
        padded_attention_masks = torch.tensor(attention_masks)

        return input_ids_resampled, padded_attention_masks, y_resampled
    
    if test_data:
        return input_ids, attention_masks

    return input_ids, attention_masks, y_label


In [ ]:

# Set batch size
# Ex: We have a batch size of 100 with a max length of 100 tokens. 100 tweets with a max of 100 words (tokens) per tensor
batch_size = 100

# Preprocess training, validation, & testing data
input_ids_train, attention_masks_train, y_train = preprocess_data(train_data, XLNetTokenizer, apply_smote=True)
input_ids_val, attention_masks_val, y_val = preprocess_data(val_data, XLNetTokenizer)
input_ids_test, attention_masks_test = preprocess_data(test_data, XLNetTokenizer, test_data=True)

# Create DataLoader for training data
train_data = TensorDataset(input_ids_train, attention_masks_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Create DataLoader for validation data
val_data = TensorDataset(input_ids_val, attention_masks_val, y_val)
val_loader = DataLoader(val_data, batch_size=batch_size)

# Create DataLoader for test data
test_data = TensorDataset(input_ids_test, attention_masks_test)
test_loader = DataLoader(test_data, batch_size=batch_size)


In [ ]:

# Define the XLNet-based model, num_labels = (1, 0) = 2
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)  

# Define optimizer and learning rate: .00002
optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f"Found GPU at index {device}: {torch.cuda.get_device_name(device)}")
else:
    device = 'cpu'
    print("No GPU devices found. Make sure your GPU is properly configured.")


In [ ]:

### > Training loop
num_epochs = 10

# Send model to gpu or cpu
model.to(device)

# Create a TensorBoard writer so that after model training we can 
# visualize metrics
writer = SummaryWriter(log_dir='logs')

''' Training loop:
 1. Forward Pass: pass a batch through the model to get predictions
 2. Compute Loss: calculate the loss between the true labels and predictions
 3. Backward Pass: use the loss to compute gradients for all model params
                   w.r.t the loss '''

# Start the model training
for epoch in range(num_epochs):

    # Set model to 'train mode'
    model.train()

    # Instantiate loss variable
    total_loss = 0

    # Loop through batches of 100 tweets stored in the training loader
    for batch in train_loader:
        
        # Unpack the batch 'loop-variable'
        input_ids, attention_masks, labels = batch

        # Send unpacked batch's constituent parts to either cpu or gpu
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        # Reset all gradients to zero
        optimizer.zero_grad()

        # Forward input_ids, attention_masks, and labels through XLNet model
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)

        # Obtain the forward loss from the XLNet model: 
        # # It represents how well the models predictions match the true labels
        loss = outputs.loss

        # Obtain the backward loss from the XLNet model:
        # Is used to calculate the derivatives of the model's parameters
        # w.r.t forward loss. Calculated through back propagation
        loss.backward()

        # Minimizes the loss, and adjusts the models weights and biases
        optimizer.step()

        # Accumulates the total loss for each batch
        total_loss += loss.item()

    # Calculate the average training loss per epoch
    avg_loss = total_loss / len(train_loader)

    # Log training loss to TensorBoard
    writer.add_scalar('Loss/Train', avg_loss, epoch)

    
    
    ### > Validation loop: Monitors progress of training
    
    # Set model to 'evaluation mode'
    model.eval()

    # Container for predicted hate-speech or non-hate-speech
    y_pred_labels = []

    # Actual values for hate-speech or non-hate-speech
    y_actual_labels = []

    # Set loss value equal to zero
    val_loss = 0

    # The context manager is used to temporarily disable gradient calcs
    with torch.no_grad():

        # batch: sample of 100 tweets with 100 tokens
        for batch in val_loader:

            # Unpack each variable in each batch
            input_ids, attention_masks, labels = batch

            # Send to the device that is being used: cpu or gpu
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
            
            # Forward input_ids, attention_masks, and labels through XLNet model
            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            
            # Accumulate validation loss
            val_loss += loss.item()
            
            # Store actual labels of hate-speech or no-hate-speece (0 or 1)
            y_actual_labels.extend(labels.cpu().numpy())
            
            # Get model predictions
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            # .extend() takes the labels from y_pred_labels and casts them into a numpy array
            y_pred_labels.extend(predictions.cpu().numpy())

    # Calculate the average validation loss per epoch
    avg_val_loss = val_loss / len(val_loader)
    
    # Calculate validation accuracy and print classification report
    val_accuracy = accuracy_score(y_actual_labels, y_pred_labels)
    val_report = classification_report(y_actual_labels, y_pred_labels)
    
    # Log validation loss and accuracy to TensorBoard
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)
    
    # Print training and validation metrics
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training Loss: {avg_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(val_report)

# Close the TensorBoard writer
writer.close()


# Save the trained coefficients of the neural net as soon as validation is done
torch.save(model.state_dict(), os.getcwd())


### > Testing Loop: unseen data

# Puts model into evaluation mode
model.eval()

# list for predicted labels
y_pred_test = []

# List for actual labels
y_actual_test = []

# Set the loss to zero
test_loss = 0

with torch.no_grad():

    # Batch: 100 tweets with 100 tokens in each batch
    for batch in test_loader:

        # Unpack each variable in each batch
        input_ids, attention_masks, labels = batch

        # Send constituents to the device being used: cpu or gpu
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)
        
        # Forward input_ids, attention_masks, and labels through XLNet model
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        
        # Accumulate test loss
        test_loss += loss.item()
        
        # Store true labels
        y_actual_test.extend(labels.cpu().numpy())
        
        # Get model predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        y_pred_test.extend(predictions.cpu().numpy())

# Calculate the average test loss per epoch
avg_test_loss = test_loss / len(test_loader)

# Calculate test accuracy and print classification report
test_accuracy = accuracy_score(y_actual_test, y_pred_test)
test_report = classification_report(y_actual_test, y_pred_test)

# Log test loss and accuracy to TensorBoard
writer.add_scalar('Loss/Test', avg_test_loss, epoch)
writer.add_scalar('Accuracy/Test', test_accuracy, epoch)

# Print test metrics
print("Test Loss:", avg_test_loss)
print("Test Accuracy:", test_accuracy)
print(test_report)


In [ ]:
%load_ext tensorboard

In [ ]:
# Run this in powershell, then open up localhost:6006 in webbrowser

 tensorboard --logdir=C:/Users/andre/'Job Portfolio Projects'/DataGlacierVI/sentiment.analysis/logs --port=6006